In [ ]:
%pwd

'/home/hajibagher/insect-classification-deeplearning'

In [1]:
from pprint import pprint
import torch
from torch import nn
import torchviz

import vit
import scipy.io
from matplotlib import pyplot as plt
import numpy as np
import dnaencoder
import pandas as pd
from importlib import reload

In [2]:
from mat import mat

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
print("Device", device)

Device cuda


In [5]:
print(mat['val_seen_loc'].shape)
print(mat['val_unseen_loc'].shape)
print(mat['test_seen_loc'].shape)
print(mat['test_unseen_loc'].shape)

(1, 3234)
(1, 3721)
(1, 4990)
(1, 7440)


In [ ]:
dict(dnaencoder.evaluate_model(mat, "amintehrani/dnaencoder-insects-finetuned"))

In [ ]:
model_name = "amintehrani/vit-insects-finetuned7"
dict(vit.evaluate_model(mat, model_name, device))

In [6]:
all_images = mat['all_images']
all_labels = mat['all_labels'].squeeze()
train_indices = mat['train_loc'].squeeze()
val_seen_indices = mat['val_seen_loc'].squeeze()
val_unseen_indices = mat['val_unseen_loc'].squeeze()
test_seen_indices = mat['test_seen_loc'].squeeze()
test_unseen_indices = mat['test_unseen_loc'].squeeze()

val_indices = np.concatenate((val_seen_indices, val_unseen_indices))
test_indices = np.concatenate((test_seen_indices, test_unseen_indices))

total_images = all_images.shape[0]
print(train_indices.shape[0], val_seen_indices.shape[0], val_unseen_indices.shape[0], test_seen_indices.shape[0], test_unseen_indices.shape[0])



13039 3234 3721 4990 7440


In [7]:
from multimodal_dataset import MultiModalDataset
from load_embeddings import load_dna_embeddings, load_img_embeddings

all_dna_features = load_dna_embeddings()
all_image_features = load_img_embeddings()

all_dna_len = list(map(lambda s: len(s.strip()), mat['all_string_dnas']))
dna_str_len_mapping: dict[int,int] = {}

def dna_str_len_to_int(s_len):
    if s_len not in dna_str_len_mapping:
        dna_str_len_mapping[s_len] = len(dna_str_len_mapping)
    return dna_str_len_mapping[s_len]

# def all_dna_len_token():
#     return list(map(dna_str_len_to_int, all_dna_len))

all_dna_len_tokens = list(map(dna_str_len_to_int, all_dna_len))
all_dna_len_tokens = np.array(all_dna_len_tokens, dtype=np.int64)

species2genus = mat['species2genus']-1


genus_species = dict()
max_specie_in_genus = 0
for genus_id, genus in pd.DataFrame(species2genus, columns=['genus']).groupby('genus'):
    specie_indices = genus.index.tolist()
    genus_species[genus_id] = specie_indices
    if len(specie_indices) > max_specie_in_genus:
        max_specie_in_genus = len(specie_indices)

import multimodal_dataset
reload(multimodal_dataset)
from multimodal_dataset import MultiModalDataset
datasets = {
    indicies_type:MultiModalDataset(
        mat['all_string_dnas'][(inds:=mat[indicies_type].flatten())],
        mat['all_images'][inds],
        np.transpose(mat['all_labels'], (1,0))[inds],
        dna_str_len_mapping,
        species2genus,
        genus_species,
        None,
        None,
        dna_embeddings=all_dna_features[inds],
        img_embeddings=all_image_features[inds]
    )
    for indicies_type in ["val_seen_loc", "val_unseen_loc", "test_seen_loc", "test_unseen_loc"]
}

Loading file: batch_dna_0.pkl
Loading file: batch_dna_1.pkl
Loading file: batch_dna_2.pkl
Loading file: batch_dna_3.pkl
Loading file: batch_dna_4.pkl
Loading file: batch_dna_5.pkl
Loading file: batch_dna_6.pkl
Loading file: batch_dna_7.pkl
Loading file: batch_dna_8.pkl
Loading file: batch_dna_9.pkl
Loading file: batch_dna_10.pkl
Loading file: batch_dna_11.pkl
Loading file: batch_dna_12.pkl
Loading file: batch_dna_13.pkl
Loading file: batch_dna_14.pkl
Loading file: batch_dna_15.pkl
Loading file: batch_dna_16.pkl
Loading file: batch_dna_17.pkl
Loading file: batch_dna_18.pkl
Loading file: batch_dna_19.pkl
Loading file: batch_dna_20.pkl
Loading file: batch_dna_21.pkl
Loading file: batch_dna_22.pkl
Loading file: batch_dna_23.pkl
Loading file: batch_dna_24.pkl
Loading file: batch_dna_25.pkl
Loading file: batch_dna_26.pkl
Loading file: batch_dna_27.pkl
Loading file: batch_dna_28.pkl
Loading file: batch_dna_29.pkl
Loading file: batch_dna_30.pkl
Loading file: batch_dna_31.pkl
Loading file: batc

Loading file: batch_img_0.pkl
Loading file: batch_img_1.pkl
Loading file: batch_img_2.pkl
Loading file: batch_img_3.pkl
Loading file: batch_img_4.pkl
Loading file: batch_img_5.pkl
Loading file: batch_img_6.pkl
Loading file: batch_img_7.pkl
Loading file: batch_img_8.pkl
Loading file: batch_img_9.pkl
Loading file: batch_img_10.pkl
Loading file: batch_img_11.pkl
Loading file: batch_img_12.pkl
Loading file: batch_img_13.pkl
Loading file: batch_img_14.pkl
Loading file: batch_img_15.pkl
Loading file: batch_img_16.pkl
Loading file: batch_img_17.pkl
Loading file: batch_img_18.pkl
Loading file: batch_img_19.pkl
Loading file: batch_img_20.pkl
Loading file: batch_img_21.pkl
Loading file: batch_img_22.pkl
Loading file: batch_img_23.pkl
Loading file: batch_img_24.pkl
Loading file: batch_img_25.pkl
Loading file: batch_img_26.pkl
Loading file: batch_img_27.pkl
Loading file: batch_img_28.pkl
Loading file: batch_img_29.pkl
Loading file: batch_img_30.pkl
Loading file: batch_img_31.pkl
Loading file: batc

In [15]:
# Evaluate:
import models
reload(models)
from models import AttentionFusion, GenusClassifier, LocalSpecieClassfier, MainClassifier, multimodal_collector

fusion_embedder = AttentionFusion(dna_dim=512,img_dim=768,dna_len_dim=32, fused_dim=258, proj_dna_dim=128-32, proj_img_dim=128, dropout=0.2)
# print("fusion_embedder", fusion_embedder._config)
# print("Fusion model created. fused dim: ", fusion_embedder.fused_dim)
genus_classifier = GenusClassifier(fusion_embedder.fused_dim,dropout=0.1, dna_len_dim=32)
local_specie_classifier = LocalSpecieClassfier(fusion_embedder.fused_dim,reduced_fused_dim=128, specie_decoder_hidden_dim=256, dropout=0.1,dna_len_dim=32)

# main_classifier = MainClassifier(mat['species2genus'], genus_species, None, None, fusion_embedder, genus_classifier,local_specie_classifier,alpha=2, beta=0, theta=0,).to(device)
main_classifier = MainClassifier.load_model("./results_20250827T234918-final", mat['species2genus'], genus_species).to(device)

assert main_classifier.fusion_embedder._config == fusion_embedder._config
assert main_classifier.genus_classifier._config == genus_classifier._config
assert main_classifier.local_specie_classifier._config == local_specie_classifier._config
# print(main_classifier)
dict(main_classifier.evaluate(datasets))


# print(main_classifier)
# main_classifier.load_model("results_20250819T030148-final")
# train_indices = (mat['train_loc'] - 1).flatten()  # Get train indices
# train_dataset = MultiModalDataset(mat['all_string_dnas'][train_indices], mat['all_images'][train_indices], np.transpose(mat['all_labels'], (1,0))[train_indices], dna_str_len_mapping, species2genus, genus_species, None, None, dna_embeddings=all_dna_features[train_indices], img_embeddings=all_image_features[train_indices])
# main_classifier.fit(
#     train_dataset,
#     None,
#     batch_size=64,
#     epochs=1,
#     eval_steps=200,
#     save_steps=400,
#     lr=1e-3
# )

In [ ]:

# main_classifier2 = MainClassifier.load_modelV2("./model_trained_v2_20250828T164138-final", mat['species2genus'], genus_species).to(device)

# pprint(dict(main_classifier2.evaluate(datasets)))

In [ ]:
# main_classifier2 = MainClassifier.load_model("./model_trained_20250828T200842-final", mat['species2genus'], genus_species).to(device)

# pprint(dict(main_classifier2.evaluate(datasets)))

In [14]:
import models
reload(models)
from models import MainClassifier


main_classifier2 = MainClassifier.load_model("model_trained_20250828T221231_freezegenus-final", mat['species2genus'], genus_species).to(device)

pprint(dict(main_classifier2.evaluate(datasets, only_genus=True)))
# pprint(dict(main_classifier2.evaluate_genus(datasets)))

Loaded model config: {'fusion_embedder': {'dna_dim': 512, 'img_dim': 768, 'fused_dim': 256, 'dna_len_dim': 32, 'num_distinct_dna_len': 120, 'proj_dna_dim': 96, 'proj_img_dim': 128, 'dropout': 0.2}, 'genus_classifier': {'fused_dim': 256, 'hidden_dim': 744, 'genus_n_classes': 372, 'dropout': 0.2, 'dna_len_dim': 32}, 'local_specie_classifier': {'fused_dim': 256, 'genus_n_classes': 372, 'reduced_fused_dim': 256, 'max_specie_in_genus': 23, 'genus_embedding_dim': 64, 'specie_decoder_hidden_dim': 256, 'dropout': 0.2, 'dna_len_dim': 32}}
Loaded model state: <All keys matched successfully>
Evaluating val_seen_loc...


/home/hajibagher/insect-classification-deeplearning/multimodal_dataset.py:89: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  res['dna_emb'] = torch.tensor(self.dna_embeddings[idx], dtype=torch.float32)
/home/hajibagher/insect-classification-deeplearning/multimodal_dataset.py:93: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  res['img_emb'] = torch.tensor(self.img_embeddings[idx], dtype=torch.float32)
/opt/conda/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Evaluating val_unseen_loc...


/home/hajibagher/insect-classification-deeplearning/multimodal_dataset.py:89: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  res['dna_emb'] = torch.tensor(self.dna_embeddings[idx], dtype=torch.float32)
/home/hajibagher/insect-classification-deeplearning/multimodal_dataset.py:93: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  res['img_emb'] = torch.tensor(self.img_embeddings[idx], dtype=torch.float32)
/opt/conda/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Evaluating test_seen_loc...


/home/hajibagher/insect-classification-deeplearning/multimodal_dataset.py:89: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  res['dna_emb'] = torch.tensor(self.dna_embeddings[idx], dtype=torch.float32)
/home/hajibagher/insect-classification-deeplearning/multimodal_dataset.py:93: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  res['img_emb'] = torch.tensor(self.img_embeddings[idx], dtype=torch.float32)
/opt/conda/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Evaluating test_unseen_loc...


/home/hajibagher/insect-classification-deeplearning/multimodal_dataset.py:89: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  res['dna_emb'] = torch.tensor(self.dna_embeddings[idx], dtype=torch.float32)
/home/hajibagher/insect-classification-deeplearning/multimodal_dataset.py:93: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  res['img_emb'] = torch.tensor(self.img_embeddings[idx], dtype=torch.float32)
/opt/conda/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


{'test_seen_loc': {'eval_accuracy': 0.5783567134268537,
                   'eval_loss': 5.358677387237549,
                   'eval_runtime': 2.4772,
                   'eval_samples_per_second': 2014.343,
                   'eval_steps_per_second': 15.743},
 'test_unseen_loc': {'eval_accuracy': 0.37446236559139784,
                     'eval_loss': 5.560892581939697,
                     'eval_runtime': 3.2243,
                     'eval_samples_per_second': 2307.49,
                     'eval_steps_per_second': 18.299},
 'val_seen_loc': {'eval_accuracy': 0.6014223871366728,
                  'eval_loss': 5.335085391998291,
                  'eval_runtime': 1.475,
                  'eval_samples_per_second': 2192.557,
                  'eval_steps_per_second': 17.627},
 'val_unseen_loc': {'eval_accuracy': 0.4254232733136254,
                    'eval_loss': 5.513214588165283,
                    'eval_runtime': 1.5053,
                    'eval_samples_per_second': 2472.013,
         

In [15]:

import models
reload(models)
from models import MainClassifier


main_classifier2 = MainClassifier.load_model("./model_trained_20250828T224831-final", mat['species2genus'], genus_species).to(device)

pprint(dict(main_classifier2.evaluate(datasets, only_genus=True)))
pprint(dict(main_classifier2.evaluate(datasets, only_genus=False)))
# pprint(dict(main_classifier2.evaluate_genus(datasets)))


Loaded model config: {'fusion_embedder': {'dna_dim': 512, 'img_dim': 768, 'fused_dim': 256, 'dna_len_dim': 32, 'num_distinct_dna_len': 120, 'proj_dna_dim': 96, 'proj_img_dim': 128, 'dropout': 0.2}, 'genus_classifier': {'fused_dim': 256, 'hidden_dim': 744, 'genus_n_classes': 372, 'dropout': 0.2, 'dna_len_dim': 32}, 'local_specie_classifier': {'fused_dim': 256, 'genus_n_classes': 372, 'reduced_fused_dim': 256, 'max_specie_in_genus': 23, 'genus_embedding_dim': 64, 'specie_decoder_hidden_dim': 256, 'dropout': 0.2, 'dna_len_dim': 32}}
Loaded model state: <All keys matched successfully>
Evaluating val_seen_loc...


/home/hajibagher/insect-classification-deeplearning/multimodal_dataset.py:89: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  res['dna_emb'] = torch.tensor(self.dna_embeddings[idx], dtype=torch.float32)
/home/hajibagher/insect-classification-deeplearning/multimodal_dataset.py:93: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  res['img_emb'] = torch.tensor(self.img_embeddings[idx], dtype=torch.float32)
/opt/conda/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Evaluating val_unseen_loc...


/home/hajibagher/insect-classification-deeplearning/multimodal_dataset.py:89: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  res['dna_emb'] = torch.tensor(self.dna_embeddings[idx], dtype=torch.float32)
/home/hajibagher/insect-classification-deeplearning/multimodal_dataset.py:93: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  res['img_emb'] = torch.tensor(self.img_embeddings[idx], dtype=torch.float32)
/opt/conda/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Evaluating test_seen_loc...


...

Evaluating test_seen_loc...


/home/hajibagher/insect-classification-deeplearning/multimodal_dataset.py:89: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  res['dna_emb'] = torch.tensor(self.dna_embeddings[idx], dtype=torch.float32)
/home/hajibagher/insect-classification-deeplearning/multimodal_dataset.py:93: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  res['img_emb'] = torch.tensor(self.img_embeddings[idx], dtype=torch.float32)
/opt/conda/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Evaluating test_unseen_loc...


/home/hajibagher/insect-classification-deeplearning/multimodal_dataset.py:89: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  res['dna_emb'] = torch.tensor(self.dna_embeddings[idx], dtype=torch.float32)
/home/hajibagher/insect-classification-deeplearning/multimodal_dataset.py:93: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  res['img_emb'] = torch.tensor(self.img_embeddings[idx], dtype=torch.float32)
/opt/conda/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


{'test_seen_loc': {'eval_accuracy': 0.30801603206412825,
                   'eval_loss': 29.34588623046875,
                   'eval_runtime': 3.2928,
                   'eval_samples_per_second': 1515.412,
                   'eval_steps_per_second': 11.844},
 'test_unseen_loc': {'eval_accuracy': 0.014381720430107527,
                     'eval_loss': 36.8821907043457,
                     'eval_runtime': 4.7767,
                     'eval_samples_per_second': 1557.558,
                     'eval_steps_per_second': 12.352},
 'val_seen_loc': {'eval_accuracy': 0.36023500309214596,
                  'eval_loss': 28.06710433959961,
                  'eval_runtime': 2.0338,
                  'eval_samples_per_second': 1590.144,
                  'eval_steps_per_second': 12.784},
 'val_unseen_loc': {'eval_accuracy': 0.009674818597151304,
                    'eval_loss': 36.51240539550781,
                    'eval_runtime': 2.5073,
                    'eval_samples_per_second': 1484.041,
   

In [12]:
import models
reload(models)
from models import MainClassifier


main_classifier2 = MainClassifier.load_model("model_trained_20250828T224022-final", mat['species2genus'], genus_species).to(device)

pprint(dict(main_classifier2.evaluate(datasets, only_genus=True)))
pprint(dict(main_classifier2.evaluate(datasets, only_genus=False)))
# pprint(dict(main_classifier2.evaluate_genus(datasets)))

Loaded model config: {'fusion_embedder': {'dna_dim': 512, 'img_dim': 768, 'fused_dim': 256, 'dna_len_dim': 32, 'num_distinct_dna_len': 120, 'proj_dna_dim': 96, 'proj_img_dim': 128, 'dropout': 0.2}, 'genus_classifier': {'fused_dim': 256, 'hidden_dim': 744, 'genus_n_classes': 372, 'dropout': 0.2, 'dna_len_dim': 32}, 'local_specie_classifier': {'fused_dim': 256, 'genus_n_classes': 372, 'reduced_fused_dim': 256, 'max_specie_in_genus': 23, 'genus_embedding_dim': 64, 'specie_decoder_hidden_dim': 256, 'dropout': 0.2, 'dna_len_dim': 32}}
Loaded model state: <All keys matched successfully>
Evaluating val_seen_loc...


/home/hajibagher/insect-classification-deeplearning/multimodal_dataset.py:89: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  res['dna_emb'] = torch.tensor(self.dna_embeddings[idx], dtype=torch.float32)
/home/hajibagher/insect-classification-deeplearning/multimodal_dataset.py:93: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  res['img_emb'] = torch.tensor(self.img_embeddings[idx], dtype=torch.float32)
/opt/conda/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Evaluating val_unseen_loc...


/home/hajibagher/insect-classification-deeplearning/multimodal_dataset.py:89: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  res['dna_emb'] = torch.tensor(self.dna_embeddings[idx], dtype=torch.float32)
/home/hajibagher/insect-classification-deeplearning/multimodal_dataset.py:93: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  res['img_emb'] = torch.tensor(self.img_embeddings[idx], dtype=torch.float32)
/opt/conda/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Evaluating test_seen_loc...


...

Evaluating test_seen_loc...


/home/hajibagher/insect-classification-deeplearning/multimodal_dataset.py:89: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  res['dna_emb'] = torch.tensor(self.dna_embeddings[idx], dtype=torch.float32)
/home/hajibagher/insect-classification-deeplearning/multimodal_dataset.py:93: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  res['img_emb'] = torch.tensor(self.img_embeddings[idx], dtype=torch.float32)
/opt/conda/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Evaluating test_unseen_loc...


/home/hajibagher/insect-classification-deeplearning/multimodal_dataset.py:89: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  res['dna_emb'] = torch.tensor(self.dna_embeddings[idx], dtype=torch.float32)
/home/hajibagher/insect-classification-deeplearning/multimodal_dataset.py:93: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  res['img_emb'] = torch.tensor(self.img_embeddings[idx], dtype=torch.float32)
/opt/conda/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


{'test_seen_loc': {'eval_accuracy': 0.3721442885771543,
                   'eval_loss': 27.76049041748047,
                   'eval_runtime': 3.2055,
                   'eval_samples_per_second': 1556.715,
                   'eval_steps_per_second': 12.167},
 'test_unseen_loc': {'eval_accuracy': 0.018951612903225806,
                     'eval_loss': 36.282920837402344,
                     'eval_runtime': 4.8968,
                     'eval_samples_per_second': 1519.354,
                     'eval_steps_per_second': 12.049},
 'val_seen_loc': {'eval_accuracy': 0.4678416821273964,
                  'eval_loss': 25.730167388916016,
                  'eval_runtime': 2.1181,
                  'eval_samples_per_second': 1526.85,
                  'eval_steps_per_second': 12.275},
 'val_unseen_loc': {'eval_accuracy': 0.02069336199946251,
                    'eval_loss': 35.923255920410156,
                    'eval_runtime': 2.3274,
                    'eval_samples_per_second': 1598.807,
   